In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/cbchealth.txt'
dest_url = './run_data/cbc_scrapt_v2.csv'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

scripts.py:2: FutureWarning: read_table is deprecated, use read_csv instead.
  import pandas as pd


,tweetid,date,title
0,586266687948881921,Thu Apr 09 20:37:25 +0000 2015,Drugs need careful monitoring for expiry dates...
1,586266687017771008,Thu Apr 09 20:37:25 +0000 2015,Sabra hummus recalled in U.S. http://www.cbc.c...
2,586266685495214080,Thu Apr 09 20:37:24 +0000 2015,U.S. sperm bank sued by Canadian couple didn't...
3,586226316820623360,Thu Apr 09 17:57:00 +0000 2015,Manitoba pharmacists want clampdown on Tylenol...
4,586164344452354048,Thu Apr 09 13:50:44 +0000 2015,Mom of 7 'spooked' by vaccinations reverses st...


In [3]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3741 entries, 0 to 3740
Data columns (total 3 columns):
tweetid    3741 non-null int64
date       3741 non-null object
title      3741 non-null object
dtypes: int64(1), object(2)
memory usage: 87.8+ KB


In [4]:
# -------------------------------------------------
frame.shape

(3741, 3)

In [5]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://www.cbc.ca/news/health/drugs-need-careful-monitoring-for-expiry-dates-pharmacists-say-1.3026749?cmp
http://www.cbc.ca/news/health/sabra-hummus-recalled-in-u-s-1.3026865?cmp
http://www.cbc.ca/news/canada/toronto/sperm-bank-sued-by-port-hope-couple-says-it-didn-t-verify-donor-info-1.3026909?cmp
http://www.cbc.ca/news/canada/manitoba/manitoba-pharmacists-want-clampdown-on-tylenol-1-sales-1.3025636?cmp
http://www.cbc.ca/news/canada/ottawa/tara-hills-ottawa-mom-changes-anti-vaccination-stand-before-7-kids-get-sick-1.3025592?cmp
http://www.cbc.ca/news/canada/hamilton/news/hamilton-police-send-mental-health-pros-to-the-front-lines-with-cops-1.3024000?cmp
http://www.cbc.ca/news/technology/wind-turbine-noise-linked-to-only-1-health-issue-annoyance-1.3025495?cmp
http://www.cbc.ca/news/business/new-e-cigarette-industry-says-regulations-off-the-mark-1.3024944?cmp
http://www.cbc.ca/news/health/dementia-patients-sold-unproven-brainwave-optimization-1.3025392?cmp
http://www.cbc.ca/news/canada/

http://www.cbc.ca/news/health/2-dose-hpv-vaccines-recommended-by-who-to-protect-against-cervical-cancer-1.2859029?cmp
http://www.cbc.ca/news/canada/nova-scotia/iphone-apps-creating-independence-for-people-with-vision-loss-1.2858113?cmp
http://www.cbc.ca/news/canada/new-brunswick/psychiatrist-who-lost-licence-in-n-b-nl-practising-in-alberta-1.2856306?cmp
http://www.cbc.ca/news/politics/2-reports-on-government-drug-insurance-plans-offer-2-remedies-1.2858649?cmp
http://www.cbc.ca/news/politics/lyme-disease-bill-to-be-challenged-by-infectious-disease-experts-in-senate-1.2858622?cmp
http://www.cbc.ca/news/politics/assisted-suicide-debate-moves-to-the-senate-with-proposed-new-bill-1.2857965?cmp
http://www.cbc.ca/news/canada/british-columbia/avian-flu-detected-at-2-fraser-valley-poultry-farms-1.2857930?cmp
http://www.cbc.ca/news/canada/nova-scotia/nova-scotia-doctors-sanctioned-for-overprescribing-narcotics-1.2857453?cmp
http://www.cbc.ca/news/health/ebola-dr-komba-songu-m-briwa-vows-to-come-

http://www.cbc.ca/news/health/ebola-outbreak-sierra-leone-declares-temporary-lockdown-1.2757827?cmp
http://www.cbc.ca/news/canada/ottawa/west-nile-virus-cases-down-because-of-cooler-summer-temperatures-1.2756972?cmp
http://www.cbc.ca/news/canada/british-columbia/josephine-james-no-idea-whatsoever-baby-nevaeh-was-on-the-way-1.2757043?cmp
http://www.cbc.ca/1.2755495
http://www.cbc.ca/news/health/ebola-crisis-who-discusses-experimental-drugs-1.2756813?cmp
http://www.cbc.ca/news/technology/scientists-urge-government-to-fund-basic-research-1.2756038?cmp
http://www.cbc.ca/news/canada/hamilton/hamilton-school-bans-eggs-dairy-in-class-for-child-with-severe-allergy-1.2756015?cmp
http://www.cbc.ca/news/canada/british-columbia/vancouver-school-board-pays-1-7m-over-student-s-heart-attack-1.2756057?cmp
http://www.cbc.ca/news/health/hpv-vaccine-why-boys-are-less-likely-to-get-it-1.2756037?cmp
http://www.cbc.ca/news/canada/why-canada-must-approach-ebola-outbreak-like-a-natural-disaster-1.2754828?cmp


http://www.cbc.ca/news/health/measles-outbreaks-spotlight-need-for-national-registry-1.2654402?cmp
http://www.cbc.ca/news/canada/newfoundland-labrador/medical-marijuana-warnings-issued-to-n-l-doctors-1.2654127?cmp
http://www.cbc.ca/news/health/e-cigarette-liquid-nicotine-makers-fight-to-keep-candy-flavour-names-1.2654233?cmp
http://www.cbc.ca/news/canada/nova-scotia/uncommon-birth-defect-appears-in-3-babies-in-homes-on-1-road-1.2652759?cmp
http://www.cbc.ca/news/health/the-made-in-canada-problem-facing-nuclear-medicine-1.2652667?cmp
http://www.cbc.ca/news/politics/drug-shortages-worsening-as-health-canada-starts-study-to-address-it-1.2652581?cmp
http://www.cbc.ca/news/canada/british-columbia/travel-insurance-doesn-t-pay-for-these-bank-customers-1.2651318?cmp
http://www.cbc.ca/news/technology/fitness-watch-review-how-latest-activity-trackers-measure-up-1.2649484?cmp
http://www.cbc.ca/news/canada/toronto/toronto-surgeons-could-make-history-with-canada-s-1st-hand-transplant-1.2652946?cmp


http://bit.ly/1cVb2PI
http://bit.ly/1alujDR
http://bit.ly/1d6tpMo
http://bit.ly/1k0fjVY
http://bit.ly/1k1oxBi
http://bit.ly/1d45CwG
http://bit.ly/1gfIoI2
http://bit.ly/19xNhKJ
http://bit.ly/1d3TMmd
http://bit.ly/1k0ILLA
http://bit.ly/1ggCWo4
http://bit.ly/1jZjz85
http://bit.ly/1llryaZ
http://bit.ly/19nBN9o
http://bit.ly/1cARakQ
http://bit.ly/1lf4WZM
http://bit.ly/1ldRULO
http://bit.ly/1e71LRx
http://bit.ly/19cqYaa
http://bit.ly/1fYpbKO
http://bit.ly/1e8fLu8
http://bit.ly/1fW2g2B
http://bit.ly/1jBQJdU
http://bit.ly/1fW2g2w
http://bit.ly/198gicw
http://bit.ly/1cMH3Eh
http://bit.ly/1cr6osl
http://bit.ly/1cqtQ9b
http://bit.ly/1jqTuhZ
http://bit.ly/1jqTuhW
http://bit.ly/1kqUAIB
http://bit.ly/1ceSSYZ
http://bit.ly/1kq5uyh
http://bit.ly/1joB2Xg
http://bit.ly/1fKKsr6
http://bit.ly/1c4WXgH
http://bit.ly/1c4NXYR
http://bit.ly/1fJ0Kkb
http://bit.ly/1c3TMG0
http://bit.ly/1dPAGSy
http://bit.ly/1jiWe0J
http://bit.ly/1dOkkto
http://bit.ly/JBJ5Q1
http://bit.ly/1c0wYXM
http://bit.ly/1fCUPxg
http://bit.

...

Canadian pot growers eyeing medical marijuana free market http://bit.ly/198dA7l


http://bit.ly/198dA7l
Athletes in junk food ads send kids mixed messages http://bit.ly/1clR4gv


http://bit.ly/1clR4gv
Listeria risk prompts Reser’s macaroni salad recall http://bit.ly/1bLLCAz


http://bit.ly/1bLLCAz
Canada's parkour gyms change course of outdoor sport http://bit.ly/191Sw2t


http://bit.ly/191Sw2t
Can an infection give children OCD? http://bit.ly/17CjSxq


http://bit.ly/17CjSxq
Doctor 'falsely imprisoned' McMaster cardiologist, judge rules http://bit.ly/18ZFz9f


http://bit.ly/18ZFz9f
Ottawa willing to talk assisted suicide, but won't legalize it http://bit.ly/1bBQwAf


http://bit.ly/1bBQwAf
Doctors worried about patients leaving hospitals too soon http://bit.ly/17B0qBk


http://bit.ly/17B0qBk
How 'toxic' is sugar? http://bit.ly/18VZ1ps


http://bit.ly/18VZ1ps
Breast cancer risk lowered by walking 1 hour a day http://bit.ly/1cb4eg5


http://bit.ly/1cb4eg5
'Poop' pills can treat C. diffici



http://bit.ly/1dsNnqn
Doctor shortage alleviated in Grand Falls-Windsor http://bit.ly/154TzM1


http://bit.ly/154TzM1
Marijuana most widely used illegal drug, global study finds http://bit.ly/15y5k0Y


http://bit.ly/15y5k0Y
Nearly 2,500 women exposed to risk of infection in Miramichi http://bit.ly/1aQ5BQC


http://bit.ly/1aQ5BQC
Freya-28 birth control pills recalled over faulty pack http://bit.ly/17gHDN4


http://bit.ly/17gHDN4
Antioxidant supplements can be overload, experts caution http://bit.ly/152GDWX


http://bit.ly/152GDWX
Rotavirus vaccine for babies may protect others http://bit.ly/1ce92Pa


http://bit.ly/1ce92Pa
Cronut burger illnesses at CNE came from maple bacon jam http://bit.ly/1dn9z58


http://bit.ly/1dn9z58
Cuba confirms cholera cases http://bit.ly/1dn9yOS


http://bit.ly/1dn9yOS
Canada 'way behind' on home-care help, patient advocates say http://bit.ly/1aLg4wG


http://bit.ly/1aLg4wG
Preinvasive breast cancer label may affect treatment choice http://bit.ly/178r85G


h

Suspend MS liberation therapy fund, doctors urge http://bit.ly/139fAak


http://bit.ly/139fAak
Ontario said to be squandering money, care for blood tests http://bit.ly/16y6c2i


http://bit.ly/16y6c2i
Tainted blood scandal stageplay holds message for today http://bit.ly/13UaWTU


http://bit.ly/13UaWTU
Report urges coverage of smoking cessation products http://bit.ly/138g80i


http://bit.ly/138g80i
Friendly robot helps kids brave needles http://bit.ly/137Ud9q


http://bit.ly/137Ud9q
Delaying umbilical cord clamping better for baby: McMaster expert http://bit.ly/13TwdgC


http://bit.ly/13TwdgC
More genes linked to autism with advanced sequencing tool http://bit.ly/1avcWWg


http://bit.ly/1avcWWg
Bans on smoking in cars with kids deemed a success http://bit.ly/1b42MLB


http://bit.ly/1b42MLB
New medical marijuana shop selling to non-certified clients http://bit.ly/1atK40F


http://bit.ly/1atK40F
How cancer and Alzheimer's disease act as aging opposites http://bit.ly/1atK3Kf


http://bit.ly


http://bit.ly/11DJ5pS
Recall affects 5 drugs on Canadian market http://bit.ly/17ywsQl


http://bit.ly/17ywsQl
Diabetes in Chinese-Canadians increases rapidly http://bit.ly/17yj7HO


http://bit.ly/17yj7HO
Few flights diverted for medical emergencies http://bit.ly/15aMxax


http://bit.ly/15aMxax
French man with MERS may have gone 12 days without symptoms http://bit.ly/14790SN


http://bit.ly/14790SN
Abortion activist Dr. Henry Morgentaler dies http://bit.ly/11z9G7E


http://bit.ly/11z9G7E
Liver cancer on the rise in Canada http://bit.ly/17qsysN


http://bit.ly/17qsysN
Smartphones create disruptions in medicine http://bit.ly/1553Nhx


http://bit.ly/1553Nhx
New mental health crisis centre opens in Winnipeg http://bit.ly/17nhtIV


http://bit.ly/17nhtIV
Nicholson touts mental-health crime bill ahead of vote http://bit.ly/153oAC2


http://bit.ly/153oAC2
New H7N9 bird flu virus easily develops drug resistance http://bit.ly/17mD8AW


http://bit.ly/17mD8AW
Company that mixed diluted chemo not i



http://bit.ly/13hrXUg


In [6]:
frame.head()

,tweetid,date,title,url
0,586266687948881921,Thu Apr 09 20:37:25 +0000 2015,Drugs need careful monitoring for expiry dates...,http://www.cbc.ca/news/health/drugs-need-caref...
1,586266687017771008,Thu Apr 09 20:37:25 +0000 2015,Sabra hummus recalled in U.S. =rss,http://www.cbc.ca/news/health/sabra-hummus-rec...
2,586266685495214080,Thu Apr 09 20:37:24 +0000 2015,U.S. sperm bank sued by Canadian couple didn't...,http://www.cbc.ca/news/canada/toronto/sperm-ba...
3,586226316820623360,Thu Apr 09 17:57:00 +0000 2015,Manitoba pharmacists want clampdown on Tylenol...,http://www.cbc.ca/news/canada/manitoba/manitob...
4,586164344452354048,Thu Apr 09 13:50:44 +0000 2015,Mom of 7 'spooked' by vaccinations reverses st...,http://www.cbc.ca/news/canada/ottawa/tara-hill...


In [7]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3741 entries, 0 to 3740
Data columns (total 4 columns):
tweetid    3741 non-null int64
date       3741 non-null object
title      3741 non-null object
url        3741 non-null object
dtypes: int64(1), object(3)
memory usage: 117.0+ KB


In [8]:
# -------------------------------------------------
frame_urls = frame['url'].values

In [ ]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//div[@class="story"]').extract()
            contents = join_txt(contents)
            print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath('').extract()     
#             print(len(tags))
            # Author
            author = xpath_selector.xpath('//*[@class="authorText"]/a/text()').extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': '','author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame.loc[key,k_] = v_
            print("====================\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

Header:   200
http://www.cbc.ca/news/health/drugs-need-careful-monitoring-for-expiry-dates-pharmacists-say-1.3026749?cmp
4321
0
TItle:  True
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  4321
tags  :  0
author  :  0



scripts.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Find all possible hastags


Header:   200
http://www.cbc.ca/news/health/sabra-hummus-recalled-in-u-s-1.3026865?cmp
1426
0
contents  :  1426
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/canada/toronto/sperm-bank-sued-by-port-hope-couple-says-it-didn-t-verify-donor-info-1.3026909?cmp
4615
0
contents  :  4615
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/canada/manitoba/manitoba-pharmacists-want-clampdown-on-tylenol-1-sales-1.3025636?cmp
7811
0
contents  :  7811
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/canada/ottawa/tara-hills-ottawa-mom-changes-anti-vaccination-stand-before-7-kids-get-sick-1.3025592?cmp
3632
0
contents  :  3632
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/canada/hamilton/news/hamilton-police-send-mental-health-pros-to-the-front-lines-with-cops-1.3024000?cmp
15645
13
contents  :  15645
tags  :  0
author  :  13

Header:   200
http://www.cbc.ca/news/technology/wind-turbine-noise-linked-to-only-1-health-issue-annoyance-1.3025495?cmp

Header:   200
http://www.cbc.ca/news/canada/windsor/aphria-medical-marijuana-to-double-its-leamington-operation-1.3019046?cmp
3861
0
contents  :  3861
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/canada/new-brunswick/herbicide-chemical-glyphosate-probably-carcinogenic-who-agency-finds-1.3018545?cmp
4436
0
contents  :  4436
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/canada/saskatchewan/saskatchewan-senior-dies-after-eating-detergent-pods-1.3018723?cmp
4374
0
contents  :  4374
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/health/genetic-tests-may-affect-insurance-employment-1.3018086?cmp
7963
0
contents  :  7963
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/health/more-tv-time-may-raise-type-2-diabetes-risk-1.3018506?cmp
2021
0
contents  :  2021
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/health/suicide-prevention-efforts-after-1st-attempt-must-last-longer-new-research-suggests-1.3017652?cmp
3963
0
cont

Header:   200
http://www.cbc.ca/news/canada/north/new-test-speeds-up-tuberculosis-diagnosis-in-nunavut-study-finds-1.3006334?cmp
3047
0
contents  :  3047
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/health/medication-reviews-may-miss-patients-who-need-them-1.3005870?cmp
10647
0
contents  :  10647
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/canada/ottawa/doctors-make-charter-challenge-on-right-to-refuse-care-on-religious-grounds-1.3006462?cmp
4182
0
contents  :  4182
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/arts/angelina-jolie-reveals-she-underwent-surgery-to-remove-ovaries-1.3006647?cmp
5647
0
contents  :  5647
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/canada/toronto/budget-freeze-forcing-ontario-hospitals-to-lay-off-nurses-ndp-says-1.3006189?cmp
2612
0
contents  :  2612
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/canada/british-columbia/3-foods-you-can-eat-after-the-best-before-date-but-might

...

Header:   200
http://www.cbc.ca/news/canada/ottawa/high-functioning-autism-creating-challenges-in-the-public-service-1.2667774?cmp
4671
12
contents  :  4671
tags  :  0
author  :  12

Header:   200
http://Canadadrugs.com
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/canada/hamilton/news/hospital-food-sucks-a-fresh-food-campaign-wants-to-hear-from-you-1.2668972?cmp
2931
0
contents  :  2931
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/health/antidepressant-drugs-may-not-be-best-treatment-robert-whitaker-1.2667410?cmp
7050
0
contents  :  7050
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/health/rare-childhood-disease-diagnosis-clues-a-comfort-to-families-1.2667829?cmp
2656
0
contents  :  2656
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/canada/montreal/quebec-end-of-life-care-law-means-new-era-for-health-providers-1.2667127?cmp
7470
0
contents  :  7470
tags  :  0
author  :  0

Header:   200
http://www.cbc.

Header:   200
http://www.cbc.ca/news/politics/ban-ki-moon-jim-yong-kim-to-address-maternal-health-summit-1.2659331?cmp
4622
0
contents  :  4622
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/politics/harper-won-t-fund-abortion-globally-because-it-s-extremely-divisive-1.2658828?cmp
4645
0
contents  :  4645
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/politics/vaccinations-are-proven-to-work-melinda-gates-stephen-harper-1.2658688?cmp
3194
0
contents  :  3194
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/health/lucky-iron-fish-in-cooking-pots-tackle-anemia-1.2658632?cmp
6148
15
contents  :  6148
tags  :  0
author  :  15

Header:   200
http://www.cbc.ca/news/politics/maternal-child-health-initiative-renewed-with-3-5b-pledge-1.2658169?cmp
4347
0
contents  :  4347
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/health/measles-in-u-s-at-record-high-level-since-1996-1.2658285?cmp
1328
0
contents  :  1328
tags  :  0
author  :  0

Hea

Header:   200
http://www.cbc.ca/news/canada/thunder-bay/mercury-poisoning-treaty-reviewed-at-who-geneva-meeting-1.2649875?cmp
4962
11
contents  :  4962
tags  :  0
author  :  11

Header:   200
http://www.cbc.ca/news/health/canadian-agency-calls-for-preventative-equipment-for-mers-1.2650602?cmp
3130
0
contents  :  3130
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/health/shorter-shifts-for-surgical-interns-may-not-benefit-patients-1.2649491?cmp
12127
14
contents  :  12127
tags  :  0
author  :  14

Header:   200
http://www.cbc.ca/news/health/orthorexia-takes-healthy-eating-to-the-extreme-1.2649838?cmp
4966
0
contents  :  4966
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/canada/calgary/big-demand-for-alberta-midwives-leads-to-long-waits-1.2649651?cmp
2641
0
contents  :  2641
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/health/obesity-to-overtake-smoking-as-worst-heart-disease-risk-factor-1.2648847?cmp
3588
0
contents  :  3588
tags  :  0
aut

Header:   200
http://www.cbc.ca/news/business/3d-printing-picks-up-slack-from-slumping-manufacturing-1.2640250?cmp
6881
13
contents  :  6881
tags  :  0
author  :  13

Header:   200
http://www.cbc.ca/news/canada/prince-edward-island/ptsd-coordinator-needed-on-p-e-i-says-psychologist-1.2640869?cmp
1225
0
contents  :  1225
tags  :  0
author  :  0

Header:   200
http://bit.ly/1sIpVcO
0
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/health/mers-outbreak-is-it-an-emergency-yet-who-will-consider-1.2640897?cmp
3588
0
contents  :  3588
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/health/red-wine-antioxidants-don-t-improve-heart-health-1.2640505?cmp
7334
15
contents  :  7334
tags  :  0
author  :  15

Header:   200
http://www.cbc.ca/news/health/binge-drinks-consumed-by-1-in-25-middle-school-kids-in-canada-1.2640296?cmp
6200
0
contents  :  6200
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/health/mers-infects-2nd-traveller-to-u-s-1.2

Header:   200
http://www.cbc.ca/news/canada/north/new-strategy-promotes-hunting-as-cure-to-nunavut-hunger-1.2632602?cmp
2828
0
contents  :  2828
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/canada/saskatoon/royal-university-hospital-adds-robot-to-care-team-1.2632577?cmp
2187
0
contents  :  2187
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/1.2630436
10663
15
contents  :  10663
tags  :  0
author  :  15

Header:   200
http://www.cbc.ca/news/health/mers-hospital-workers-in-indiana-clear-so-far-1.2632320?cmp
1987
0
contents  :  1987
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/technology/young-blood-reverses-some-impacts-of-aging-in-mice-1.2632167?cmp
10060
0
contents  :  10060
tags  :  0
author  :  0

Header:   200
http://www.cbc.ca/news/world/polio-outbreaks-a-public-health-emergency-world-health-organization-1.2632090?cmp
5857
11
contents  :  5857
tags  :  0
author  :  11

Header:   200
http://www.cbc.ca/news/world/mers-saudi-arabia-reports-1

In [ ]:
frame.head()

In [ ]:
frame.info()

In [ ]:
frame.to_csv(dest_url, sep='|', encoding='utf-8',index=False)

In [ ]:
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
frame = pd.read_csv(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame_content = pd.read_csv(dest_url, sep='|', encoding="utf-8")
### Writing content into text file
### ================================
file_names = frame['tweetid'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './training/' + str(name) + '.txt'
#     success_date = './date/success_date.txt'
    reg_special = r"([^a-zA-Z\s]+)"
    reg_dspace = r"(\s{2,})"
    
    try:
    
        # Writing on file
        contents = str(frame_content['contents'].iloc[key])
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"w+")
            
            # Clean content a bit
            contents = re.sub(reg_special, '', contents)
            contents = re.sub(reg_dspace,'',contents)
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
            
            #Write success load date
#             f1 = open(success_date, "a")
#             f1.write(frame['date'].iloc[key])
#             f1.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================